# Epic Seven Gear Simulator Testing Documentation
This notebook documents the testing process.

* The json data files are stored in the `/data` folder. The scripts to prepare these json file are also in the `/data` folder.
* The test modules (along with this notebook) are stored in the `/tests` folder.
* The class and function modules for the app are stored in the `/src` folder.
* `unittest` module is used for running the tests.

### Directory Structure Issue

Modules in `/tests` folder have to import data and modules from the `/data` or `/src` folders respectively, so we need to add the parent directory to the path.

This is solved by importing `set_directory` function from the `set_directory_function` module. As such, every testing module should have the following two lines of code at the top, so that the parent folder can be added to the path.

In [ ]:
from set_directory_function import set_directory
set_directory()

### Running the tests
The tests must be run from the root directory as:

python tests/test_module_name.py

e.g.,
python tests/test_get_stat_by_id.py

## Testing Validation Functions
In many of the functions that follow, the inputs need to be validated, so this section tests whether these validation functions work as intended.

### validate_stat_id()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Raises a ValueError input is None
* Raises a ValueError if input is not str. 
* Acceptable parameters: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 (as of now, if more stats are added this could increase later). Parameter can be entered as str or int.

Returns: str(stat_id)

Test Module: [test_validate_stat_id.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_stat_id.py)

Test Plan:
* stat_id = '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'. '10' (exactly as written in STATS file, should work)
* stat_id = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 (ints, should work)
* stat_id = -1 (some negative number, should be error)
* stat_id = 'Health' (some string that's not a number)
* stat_id = 111 (some large number as int that is not in acceptable parameter)
* stat_id = '111' (some large number as str that is not in acceptable parameter)
* stat_id = None (should raise ValueError)

Result: ALL PASSED.

### validate_gear_type()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Returns None if input is None
* Raises a ValueError if input is not str. 
* Acceptable parameters: 'Weapon', 'Helm', 'Armor', 'Necklace', 'Ring', 'Boots'. Parameters are converted to lower case before validating, so lower, upper or mixed case is acceptable.

Returns: gear_type.lower()

Test Module: [test_validate_gear_type.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_gear_type.py)

Test Plan:
* gear_type = 'Weapon', 'Helm', 'Armor', 'Necklace', 'Ring', 'Boots' (exactly as written in TYPES file, should work)
* gear_type = 'weapon', 'helm', 'armor', 'necklace', 'ring', 'boots' (all lower case should still work)
* gear_type = 'weapOn', 'hElm', 'arMor', 'nEcklacE', 'rINg', 'BOOTS' (mix of lower and upper case, should work)
* gear_type = 'Sword', 'mask', 'MAIL', 'Jewelry', 'Accessory', 'Shoes' (random strings, should raise error)
* gear_type = 1 (non string parameter, should raise ValueError)
* gear_type = None (should return None)

Results: ALL PASSED.

### validate_stat_type()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Validates stat_type input and returns stat_type.lower()
* Defaults to 'mainstat' if input is None.
* Raises a ValueError if input is not str. 
* Acceptable parameters: 'mainstat' and 'substat'. Parameters are converted to lower case before validating, so lower, upper or mixed case is acceptable.

Returns: stat_type.lower()

Test Module: [test_validate_stat_type.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_stat_type.py)

Test Plan:
* valid inputs
    * stat_type = 'mainstat', 'substat', 'Mainstat', 'Substat', 'MainSTAT', 'SUBstAt'
* invalid inputs
    * stat_type =  'mainstats', 'substats', 'subs', 'main', 'stats', 1, None

Results: ALL PASSED.

## Testing Stat Class

### get_stat_by_id()

Source: [stats.py](https://github.com/mesaqlain/e7_items/blob/main/src/stats.py)

Function: Retrieves data on a STAT based on the given id. It returns a dictionary.

Test Module: [test_get_stat_by_id.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_stat_by_id.py)

The Stat() class initiates with no data. Running the get_stat_by_id() method should return a dictionary from the STATS dictionary based on the id provided. The function should be able to handle both int and str arguments as long as the number is between 0 and 10. So we need to test the following cases.

Valid stat_id should return a dictionary containing stats where id is 0 or 7 (compare to STATS[0] and STATS[7]):
* stat_id = 0 
* stat_id = '0' 
* stat_id = 7 
* stat_id = '7' 

Invalid stat_id should raise ValueError:
* stat_id = 100
* stat_id = '100'
* stat_id = -1 (negative case)
* stat_id = 1000000 (big number)
* stat_id = 'Health' (some random string)
* stat_id = None


Result: ALL PASSED

### get_random_stat()

Function: 
* Summary: Retrieves data on a random STAT chosen from the list of STATS in the stats.json file. 
* Validates gear_type input
* Initializes an empty list called pool
* If gear_type is None, the pool will contain all stats from the stats.json file.
* If gear_type is provided, the pool will be limited to the stats allowed according to stat_type (for more information on allowed stats on specific gear type, refer to prep_data_TYPES.py module or check Section 5.2 in the README file.)
* Chooses a random id from the pool and runs the get_stat_by_id() function to get that stat.

Returns: Stat dic

Test Module: [test_get_random_stat.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_random_stat.py)

Test Plan:
* get_random_stat() with no inputs (this means it should use 'mainstat' as default gear_type = None)
    * Check that we do get a dictionary that is one of the stats entries in stats.json PASSED 
    * Generate 1000 random stats and count how many times each stat was chosen, they should all have been chosen at least once. PASSED

* Valid stat_type inputs (while gear_type = None):
    * Repeat above test by specifying stat_type = 'mainstat'. PASSED
    * Repeat above test by specifying stat_type = 'substat'. PASSED

* Valid gear_type inputs (while stat_type = None):
    * Repeat above test by specifying gear_type = 'weapon' (we expect to see the stat_id = 0 only (because Weapons can only have that as mainstat)) PASSED
    * Repeat above test by specifying gear_type = 'helm' (we expect to see the stat_id = 2 only (because Helms can only have that as mainstat)) PASSED
    * Repeat above test by specifying gear_type = 'armor' (we expect to see the stat_id = 4 only (because Armor can only have that as mainstat)) PASSED
    * Repeat above test by specifying gear_type = 'necklace' (we expect to see no stat_id = 8,9,10 (because Necklace cannot have those as mainstat), while others should be chosen at least once)  PASSED
    * Repeat above test by specifying gear_type = 'ring' (we expect to see no stat_id = 6,7,10 (because Ring cannot have those as mainstat), while others should be chosen at least once) PASSED
    * Repeat above test by specifying gear_type = 'boots' (we expect to see no stat_id = 6,7,8,9 (because Boots cannot have those as mainstat), while others should be chosen at least once) PASSED
    
* Valid gear_type inputs (while stat_type = 'mainstat'):
    * Repeat all tests in the previous section by specifying stat_type = 'mainstat', we should see same results ALL PASSED

* Valid gear_type inputs (while stat_type = 'substat'):
    * Repeat above test by specifying gear_type = 'weapon' (we expect to see no stat_id = 1, 4, 5 (because Weapons cannot have those as substats), while others should be chosen at least once) PASSED
    * Repeat above test by specifying gear_type = 'helm' (we expect to see no stat_id = 2 (because Helms cannot have those as substats), while others should be chosen at least once) PASSED
    * Repeat above test by specifying gear_type = 'armor' (we expect to see no stat_id = 0, 1, 4 (because Armors cannot have those as substats), while others should be chosen at least once) PASSED
    * Repeat above test by specifying gear_type = 'necklace', 'ring', 'boots' (we expect see all stats chosen at least once) PASSED
    
* Invalid stat_type inputs (while gear_type = None):
    * stat_type = 111, '111', 'health' (test integers, random strings, etc) PASSED
    
* Invalid gear_type inputs (while stat_type = None):
    * gear_type = 111, '1', 'Sword' (test integers, random strings, etc) PASSED

* Invalid inputs for both:
    * gear_type = 111, stat_type = 'Sword' PASSED

Test Plan Implementation:
* Create an empty set called selected_stat_ids = set()
* Generate 1000 random stats and take the 'id' of those stats and add it to selected_stat_ids
* Define expected_stat_ids, which contains the allowed stats for a particular gear type (based on stats)
* Calculate missing_stat_ids = expected_stat_ids - selected_stat_ids
* Calculate extra_stat_ids = selected_stat_ids - expected_stat_ids
* If missing_stat_ids set is empty, it means that all the expected stats were selected, and the assertion will pass.
* If extra_stat_ids set is empty, it means that no extra stats were selected, and the assertion will pass.

Results: ALL PASSED